In [71]:
from llama_index import Document, VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index.storage.storage_context import StorageContext
# from transformers import AutoTokenizer, AutoModel
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
import chromadb
import pandas as pd 
import openai
import os
import getpass

In [3]:
data_path = os.path.join('/workspace/data/')
index_path = os.path.join('/workspace/db/local')

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


In [5]:
from llama_index import Document, VectorStoreIndex 
from llama_index.node_parser import SentenceSplitter

In [72]:
data = pd.read_csv(os.path.join(data_path, 'financial', 'SHINHAN BANK_Financial_Product_Scraping_Result_20240104_LOAN_vec.csv'))
data.head(1)

,대출_고유키,금융상품명,상품특징,대출신청자격,상품설명
0,대출_0,혁신성장산업 지원자금,9대 선도산업 등 정부발표 혁신성장 분야에 대한 효율적 금융지원,○ (지원대상) 다음 중 하나에 해당하는 기업 혁신성장 정책금융협의회 선정 「혁신성...,9대 선도산업 등 정부발표 혁신성장 분야에 대한 효율적 금융지원


In [73]:
category = 'loan'

id_list = data.대출_고유키.values.tolist()
name_list = data.금융상품명.values.tolist()
desc_list = data.대출신청자격.values.tolist()

In [75]:
documents = []
for idx in range(len(desc_list)):
    doc = Document(text=desc_list[idx], doc_id=id_list[idx], metadata={"category": category, "name": name_list[idx]}, \
                   excluded_llm_metadata_keys = ['category', 'name'])
    documents.append(doc)

In [76]:
print(f'embedding model see this: {documents[2].get_content(metadata_mode=MetadataMode.EMBED)}', end='\n\n')
print(f'LLM see this: {documents[2].get_content(metadata_mode=MetadataMode.LLM)}')

embedding model see this: category: loan
name: 동산·채권 담보대출 지원자금

○ (지원대상) 동산채권 등의 담보에 관한 법률에 따라 동산채권 등의 선순위 근담보권(한정근, 특정근)설정 가능한 기업 * 중소중견기업 또는 상업등기법에 의거 상호등기한 업력 1년 이상 개인사업자○ (자금용도) 설비투자, R&D, 운영자금 등 * 주식 인수 등 지분투자 또는 영업 양수도를 위한 자금지원 제외 * 지원대상에 관한 추가 상세사항은 가까운 영업점 앞 문의

LLM see this: ○ (지원대상) 동산채권 등의 담보에 관한 법률에 따라 동산채권 등의 선순위 근담보권(한정근, 특정근)설정 가능한 기업 * 중소중견기업 또는 상업등기법에 의거 상호등기한 업력 1년 이상 개인사업자○ (자금용도) 설비투자, R&D, 운영자금 등 * 주식 인수 등 지분투자 또는 영업 양수도를 위한 자금지원 제외 * 지원대상에 관한 추가 상세사항은 가까운 영업점 앞 문의


In [77]:
from llama_index.node_parser import SentenceSplitter 

In [78]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)   # SentenceSplitter(chunk_size=1024, chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents)

In [79]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name, embed_batch_size=32)

In [80]:
from llama_index import ServiceContext
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores import SimpleVectorStore
from llama_index.storage.index_store import SimpleIndexStore

In [81]:
persist_dir=os.path.join(index_path, 'demo', 'qualification')

In [82]:
service_context = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model)
# storage_context = StorageContext.from_defaults(persist_dir=persist_dir)

In [83]:
# build index 
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context #, storage_context=storage_context
)

In [84]:
index.index_id

'4ba7b949-6ee0-4b39-8c76-781eca91b9c1'

In [ ]:
index.as_query_engine().query('보증 자격').get_context()

In [85]:
# set index_id to save multiple indexes to the same folder  -> persist 시 추가되는게 아니라 덮어 쓰여짐 .. => 해결 ! sotrage_context 지정안해줘서 그럼 
# storage context.from_defaults(persist_dir ~)은 docstore.json 파일이 있어야만 지정할 수 있는데 처음 인덱스 생성시에는 해당 부분 생략해서 docstore.json 파일 먼저 생성하기
index.set_index_id('scrapping_qualification')

In [86]:
index.index_id

'scrapping_qualification'

### Index data insert

### Save Index 

In [87]:
index.storage_context.persist(persist_dir=persist_dir)

### Index Load 